In [1]:
%matplotlib tk
import numpy as np
import sys
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random as rd
import copy as cp
from datetime import datetime as dt
#
sys.path.append('..')
from PyAstroPol import *
roundOffDisplay(5)

In [2]:
class KTTCoelostat(System):
    def __init__(self, Sun, Sun_disp):
        #
        self.Sun = Sun
        Dec = np.radians(self.Sun.Declination)
        HA = np.radians(self.Sun.HourAngle)
        Lat = np.radians(self.Sun.Latitude)
        PolarAxis = np.array([0, np.sin(Lat), np.cos(Lat)])
        self.PolarAxis = PolarAxis
        #
        self.Dist_EW = 830.0  # East-West distance
        self.Dist_ZN = 740.0  # North-South distance
        #
        A = -np.nan_to_num(np.abs(HA)/HA, nan=-1.0)*self.Dist_EW
        B = self.Dist_ZN
        SunPos = np.array([np.sin(HA)*np.cos(Dec), 
                           np.cos(HA)*np.cos(Dec)*np.cos(Lat) + np.sin(Dec)*np.sin(Lat), 
                           -np.cos(HA)*np.cos(Dec)*np.sin(Lat) + np.sin(Dec)*np.cos(Lat)])
        #
        TEMP = np.dot(SunPos, PolarAxis)
        a = TEMP**2-PolarAxis[2]**2
        b = -2*B*PolarAxis[1]*PolarAxis[2]
        c = (A**2+B**2)*TEMP**2 - B**2*PolarAxis[1]**2
#         self.a, self.b, self.c = a, b, c
        if (a == 0):
            C = -c/b
        else:
            C = (-b + np.nan_to_num(np.abs(Dec)/Dec, nan=1.0)*np.nan_to_num(np.sqrt(b**2-4*a*c)))/2.0/a
        self.Dist_NS = np.abs(C)
        #
        self.Center1 = np.array([-A, 0.0, -C])
        self.Center2 = np.array([0.0, B, 0.0])
        #
        self.Incidence1 = -SunPos
        self.Reflection1 = np.array([A, B, C])/np.linalg.norm(np.array([A, B, C]))
        self.Normal1 = (self.Reflection1-self.Incidence1)/np.linalg.norm(self.Reflection1-self.Incidence1)
        self.Incidence2 = self.Reflection1
        self.Reflection2 = np.array([0.0, -1.0, 0.0])
        self.Normal2 = (self.Reflection2-self.Incidence2)/np.linalg.norm(self.Reflection2-self.Incidence2)
        #
        self.Sun.makeOrigin(self.Center1-self.Sun.Distance*self.Incidence1)
        Sun_disp.makeOrigin(self.Center1-self.Sun.Distance*self.Incidence1)
        #
        C12 = Coating([2.16-7.18j, 1.67], [0.100, 0.00])
        self.M1 = Surface(600, Mirror=True, n2=2.16-7.18j)
        self.M1.pointToDirection(self.Normal1)
        self.M1.makeOrigin(self.Center1)
        # self.M1.Coating = C12
        #
        self.M2 = Surface(600, Mirror=True, n2=2.16-7.18j)
        self.M2.pointToDirection(self.Normal2)
        self.M2.makeOrigin(self.Center2)
        # self.M2.Coating = C12
        #
        self.Window = Detector(500.0)
        self.Window.rotateAboutX(90.0)
        System.__init__(self, self.Sun, [self.M1, self.M2], self.Window, dRays=Sun_disp)
        return

In [3]:
Sun = AstroSource(10000, Clear=300, Dec=10.0, HA=-40.0, Lat=10.23, Dist=5000.0)             # Position of the Sun
Sun_disp = AstroSource(20, Clear=300, Dec=10.0, HA=-40.0, Lat=10.23, Dist=5000.0)           # For display
Coel = KTTCoelostat(Sun, Sun_disp)                                                          # Colostat configuration
Coel.propagateRays()                                                                        #  

In [4]:
Fig = plt.figure()                                             
Ax = Fig.add_subplot(111, projection='3d')
Coel.draw(Ax)
adjustAspect(Ax, 5000.0)

In [5]:
MM, Tra = Coel.getSystemMuellerMatrix()
print('Mueller matrix is : \n', MM)
print('Throughput is : ', Tra)

Mueller matrix is : 
 [[ 1.00000 -0.04786  0.04438  0.00105]
 [-0.06508  0.67794 -0.73490 -0.01715]
 [-0.00499  0.71865  0.65745  0.21693]
 [ 0.00105 -0.14820 -0.15934  0.97385]]
Throughput is :  0.7341633951470367


In [6]:
print('M1 rs : ', Coel.Components[0].rs[0,0])
print('M1 rp : ', Coel.Components[0].rp[0,0])
print('M2 rs : ', Coel.Components[1].rs[0,0])
print('M2 rp : ', Coel.Components[1].rp[0,0])
A1 = np.degrees(Coel.Components[0].CoordTheta[0,0])
A2 = np.degrees(Coel.Components[1].CoordTheta[0,0])
MM_ZMX = MuellerRotationMatrix(A2)*MM*MuellerRotationMatrix(-A1)
print('Mueller matrix to be compared with previous work : \n', MM_ZMX)

M1 rs :  (-0.9316611942445823+0.17181090297388768j)
M1 rp :  (0.8443366256955277-0.31234141535058946j)
M2 rs :  (-0.9093430229205087+0.2129836696764579j)
M2 rp :  (0.8835656107703029-0.255328527692456j)
Mueller matrix to be compared with previous work : 
 [[ 1.00000  0.06499 -0.00606  0.00105]
 [ 0.06199  0.91602 -0.39470  0.06854]
 [ 0.02041  0.40055  0.89054 -0.20653]
 [ 0.00105  0.02054  0.21664  0.97385]]
